In [ ]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch import nn

import numpy as np

In [ ]:
from common.utils import seed_everything
from train import get_dataset
from draw_utils import visualize_dataset

In [ ]:
from common.logger import logger
import logging

# DEBUG INFO WARNING ERROR CRITICAL
logger.setLevel(logging.INFO)

In [ ]:
seed_everything(1024)

# Backbone

In [ ]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

In [ ]:
backbone = resnet_fpn_backbone(
    'resnet18', # resnet18, resnet50
    pretrained=True,
    trainable_layers=5, # all layers
    # trainable_layers=3,
    returned_layers=[2,3]
)

In [ ]:
test_tensor = torch.randn(1,3,384,384)
feature_dict = backbone(test_tensor)
feature_dict.keys()

In [ ]:
features = list(feature_dict.values())

In [ ]:
for f in features:
    print(f.shape)

# PennFudanPed

In [ ]:
from dataset.penn_fudan_dataset import PennFudanDataset

In [ ]:
PENN_FUDAT_ROOT_PATH = './data/PennFudanPed'
dataset_train = PennFudanDataset(PENN_FUDAT_ROOT_PATH, train=True)

In [ ]:
img, mask, bboxs = dataset_train[0]
img.shape, mask.shape, bboxs.shape

In [ ]:
visualize_dataset(dataset_train, count=5, size=3)

# Pascal VOC

In [ ]:
from dataset.pascal_voc_dataset import PascalVOCDataset

In [ ]:
dataset_voc = PascalVOCDataset('./data/VOCdevkit/', 'TRAIN', transforms=None)

In [ ]:
len(dataset_voc)

In [ ]:
visualize_dataset(dataset_voc, count=5, size=3)

# Model

In [ ]:
from centernet import create_model

In [ ]:
model = create_model()

In [ ]:
img, mask, bboxs = dataset_train[0]
img.shape, mask.shape, bboxs.shape

In [ ]:
out = model(img)

In [ ]:
out.shape

# Test

In [ ]:
from draw_utils import make_prediction
from centernet import create_model

In [ ]:
dataset_train, dataset_test = get_dataset()

In [ ]:
model = create_model()
model_name = 'centernet_v1'
model_save_name = f'{model_name}.pth'
model.load_state_dict(torch.load(model_save_name))
logger.info(f'Model loaded from {model_save_name}')

In [ ]:
# make_prediction(model, dataset_test, index=0)

In [ ]:
make_prediction(model, dataset_train, index=0, threshold=0.5)

# Gaussian Kernel

In [ ]:
g_kernel = np.array([
    [0.0625, 0.125, 0.0625],
    [0.125, 0.25, 0.125],
    [0.0625, 0.125, 0.0625]]
)

In [ ]:
g_kernel_size = g_kernel.shape[0]
w = g_kernel_size // 2

In [ ]:
x = 1
y = 1

center_mask = np.zeros((40, 40), dtype='float32')
center_mask[y-w:y+w+1,x-w:x+w+1] = g_kernel
show_images([center_mask], ["mask"], size=4)